## Blueheart Tools
- Notebook to go through cruise folders on blueheart and do different things, e.g. if grids and/or Qimera projects exist
- Note that you need to be connected to Blueheart
- Running the bash cells doesn't work (yet), you need to copy the commands in a terminal

### Sync folders to local disk

In [ ]:
%%bash
cd /Volumes/bathymetry/_blueheart/00_MARIASMERIAN/MERIAN_GEOMAR/MSM75/raw
find . -maxdepth 4 -name '*EM122.all' -exec cp -n {} /Users/mschumacher/Docs_Data/Bathy/Processing/MSM75/EM122 \;

#### List dirs without Qimera folder (bash)

##  SONNE

In [ ]:
%%bash
cd /Volumes/bathymetry/_blueheart/00_SONNE/SONNE_GEOMAR

parent_dir=/Volumes/bathymetry/_blueheart/00_SONNE/SONNE_GEOMAR
list_dir=/Users/mschumacher/Docs_Data/Bathy/Processing

ending="_Qimera"

for dir in "$parent_dir"/*; do
    if [ -d "$dir" ]; then
        if ! find "$dir" -maxdepth 1 -type d -name "*$ending" | grep -q .; then
            echo "$dir" >> "$list_dir"/SONNE_qimera_list.txt
        fi
    fi
done

In [ ]:
%%bash

cd /Volumes/bathymetry/_blueheart/00_SONNE/SONNE_GEOMAR

rm SONNE_grid_list.txt 
rm SONNE_cruise_list.txt 
rm SONNE_zero_list.txt 
rm SONNE_gpkg_list.txt 

ls */*/_grd/*EPSG3395.tif > SONNE_grid_list.txt 
ls > SONNE_cruise_list.txt
ls */*/_grd/*_zero.tif > SONNE_zero_list.txt
ls SONNE_Cov/*_Area.gpkg > SONNE_gpkg_list.txt


ls: */*/_grd/*_area.gpkg: No such file or directory


CalledProcessError: Command 'b'\ncd /Volumes/bathymetry/_blueheart/00_SONNE/SONNE_GEOMAR\n\nrm SONNE_grid_list.txt \nrm SONNE_cruise_list.txt \nrm SONNE_zero_list.txt \nrm SONNE_gpkg_list.txt \n\nls */*/_grd/*EPSG3395.tif > SONNE_grid_list.txt \nls > SONNE_cruise_list.txt\nls */*/_grd/*_zero.tif > SONNE_zero_list.txt\nls */*/_grd/*_area.gpkg > SONNE_gpkg_list.txt\n'' returned non-zero exit status 1.

In [ ]:
import numpy as np
import os

path = '/Volumes/bathymetry/_blueheart/00_SONNE/SONNE_GEOMAR/'
cruise_list = '/Volumes/bathymetry/_blueheart/00_SONNE/SONNE_GEOMAR/SONNE_cruise_list.txt'
grid_list = '/Volumes/bathymetry/_blueheart/00_SONNE/SONNE_GEOMAR/SONNE_grid_list.txt'

with open(cruise_list, 'r') as c_fi:
    cruise_path = c_fi.readlines()
    CRUISE = []
    for c_path in cruise_path:
        cruise = c_path.split('\n')[0]
        CRUISE.append(cruise)
with open(grid_list, 'r') as g_fi:
    grid_path = g_fi.readlines()
    GRID = []
    for g_path in grid_path:
        #print(os.path.join(path,g_path))
        grid = g_path.split('/')[0]
        GRID.append(grid)

# Compare cruise lists and grid list to evaluate where grids are missing

missing_grids = [ cr for cr in CRUISE if cr not in GRID ]
print(np.array(missing_grids))


['SO164-1' 'SO164-2' 'SO283' 'SONNE_Cov' 'SONNE_cruise_list.txt'
 'SONNE_grid_list.txt' 'Sonne_Geomar_existing-datasets.txt']


### Build virtual raster tiles (vrt) and convert to geotiff to merge single files

In [ ]:
%%bash

# separate bands (! caution: very large file!)
#gdalbuildvrt -overwrite -separate -input_file_list SONNE_grid_list.txt SONNE_GEOMAR_AllSoundings_DivRes_separate_EPSG3395.vrt
#gdal_translate -of GTiff -co "COMPRESS=DEFLATE" -co "TILED=YES" -co "BIGTIFF=YES" SONNE_GEOMAR_AllSoundings_DivRes_separate_EPSG3395.vrt SONNE_GEOMAR_AllSoundings_DivRes_separate_EPSG3395.tif

# one band
gdalbuildvrt -overwrite -input_file_list SONNE_grid_list.txt SONNE_GEOMAR_AllSoundings_DivRes_EPSG3395.vrt
gdal_translate -of GTiff -co "COMPRESS=DEFLATE" -co "TILED=YES" -co "BIGTIFF=YES" SONNE_GEOMAR_AllSoundings_DivRes_EPSG3395.vrt SONNE_GEOMAR_AllSoundings_DivRes_EPSG3395.tif


##  MERIAN

In [ ]:
%%bash

cd /Volumes/bathymetry/_blueheart/00_MARIASMERIAN/MERIAN_GEOMAR

rm MERIAN_grid_list.txt 
rm MERIAN_cruise_list.txt 
rm MERIAN_zero_list.txt
rm MERIAN_gpkg_list.txt

ls */*/_grd/*EPSG3395.tif > MERIAN_grid_list.txt 
ls > MERIAN_cruise_list.txt
ls */*/_grd/*_zero.tif > MERIAN_zero_list.txt
ls MERIAN_Cov/*_Area.gpkg > MERIAN_gpkg_list.txt


In [5]:
import numpy as np
import os
from osgeo import gdal

cruise_list = '/Volumes/bathymetry/_blueheart/00_MARIASMERIAN/MERIAN_GEOMAR/MERIAN_cruise_list.txt'
grid_list = '/Volumes/bathymetry/_blueheart/00_MARIASMERIAN/MERIAN_GEOMAR/MERIAN_grid_list.txt'

with open(cruise_list, 'r') as c_fi:
    cruise_path = c_fi.readlines()
    CRUISE = []
    for c_path in cruise_path:
        cruise = c_path.split('\n')[0]
        CRUISE.append(cruise)
with open(grid_list, 'r') as g_fi:
    grid_path = g_fi.readlines()
    GRID = []
    for g_path in grid_path:
        grid = g_path.split('/')[0]
        #print(grid)
        GRID.append(grid)

# Compare cruise lists and grid list to evaluate where grids are missing
missing_grids = [ cr for cr in CRUISE if cr not in GRID ]
#print(len(missing_grids))
print(np.array(missing_grids))

['MERIAN_Cov' 'MERIAN_cruise_list.txt' 'MERIAN_grid_list.txt']


### Build virtual raster tiles (vrt) and convert to geotiff to merge single files

In [ ]:
%%bash

#gdal_merge -o METEOR.tif -n 0 -co COMPRESS=DEFLATE -co TILED=YES -co BIGTIFF=YES --optfile METEOR_grid_list.txt 

# separate bands (! caution: very large file!)
#gdalbuildvrt -overwrite -separate -input_file_list MERIAN_grid_list.txt MERIAN_GEOMAR_AllSoundings_DivRes_separate_EPSG3395.vrt
#gdal_translate -of GTiff -co "COMPRESS=DEFLATE" -co "TILED=YES" -co "BIGTIFF=YES" MERIAN_GEOMAR_AllSoundings_DivRes_separate_EPSG3395.vrt MERIAN_GEOMAR_AllSoundings_DivRes_separate_EPSG3395.tif

# one band
gdalbuildvrt -overwrite -input_file_list MERIAN_grid_list.txt MERIAN_GEOMAR_AllSoundings_DivRes_EPSG3395.vrt
gdal_translate -of GTiff -co "COMPRESS=DEFLATE" -co "TILED=YES" -co "BIGTIFF=YES" MERIAN_GEOMAR_AllSoundings_DivRes_EPSG3395.vrt MERIAN_GEOMAR_AllSoundings_DivRes_EPSG3395.tif


##  METEOR

In [ ]:
%%bash

cd  /Volumes/bathymetry/_blueheart/00_METEOR/METEOR_GEOMAR

rm METEOR_grid_list.txt 
rm METEOR_cruise_list.txt 
rm METEOR_gpkg_list.txt
rm METEOR_zero_list.txt

ls */*/_grd/*EPSG3395.tif > METEOR_grid_list.txt 
ls > METEOR_cruise_list.txt
ls */*/_grd/*_zero.tif > METEOR_zero_list.txt 
ls METEOR_Cov/*_Area.gpkg > METEOR_gpkg_list.txt 



In [ ]:
import numpy as np

cruise_list = '/Volumes/bathymetry/_blueheart/00_METEOR/METEOR_GEOMAR/METEOR_cruise_list.txt'
grid_list = '/Volumes/bathymetry/_blueheart/00_METEOR/METEOR_GEOMAR/METEOR_grid_list.txt'

with open(cruise_list, 'r') as c_fi:
    cruise_path = c_fi.readlines()
    CRUISE = []
    for c_path in cruise_path:
        cruise = c_path.split('\n')[0]
        CRUISE.append(cruise)
with open(grid_list, 'r') as g_fi:
    grid_path = g_fi.readlines()
    GRID = []
    for g_path in grid_path:
        grid = g_path.split('/')[0]
        GRID.append(grid)

# Compare cruise lists and grid list to evaluate where grids are missing
missing_grids = [ cr for cr in CRUISE if cr not in GRID ]
print(np.array(missing_grids))

5 ['M207' 'METEOR_Cov' 'METEOR_cruise_list.txt' 'METEOR_grid_list.txt'
 'Meteor_Geomar_existing-data.txt']


### Create merged grid from single grids and coverage as gpkg

In [ ]:
%%bash

#gdal_merge -o METEOR.tif -n 0 -co COMPRESS=DEFLATE -co TILED=YES -co BIGTIFF=YES --optfile METEOR_grid_list.txt 
# separate bands
#gdalbuildvrt -overwrite -separate -input_file_list METEOR_grid_list.txt METEOR_GEOMAR_AllSoundings_DivRes_separate_EPSG3395.vrt
#gdal_translate -of GTiff -co "COMPRESS=DEFLATE" -co "TILED=YES" -co "BIGTIFF=YES" METEOR_GEOMAR_AllSoundings_DivRes_separate_EPSG3395.vrt METEOR_GEOMAR_AllSoundings_DivRes_separate_EPSG3395.tif

# one band
gdalbuildvrt -overwrite -input_file_list METEOR_grid_list.txt METEOR_GEOMAR_AllSoundings_DivRes_EPSG3395.vrt
gdal_translate -of GTiff -co "COMPRESS=DEFLATE" -co "TILED=YES" -co "BIGTIFF=YES" METEOR_GEOMAR_AllSoundings_DivRes_EPSG3395.vrt METEOR_GEOMAR_AllSoundings_DivRes_EPSG3395.tif


## Polygonise geotiffs
- calculate zero grid from grid list
- create list from zero grids
- polygonise zero grids to avoid millions of features due to colour change
- create list from shape files
- remove unneccessary features in shapes (those with '0')
- dissolve fields
- add field for cruise name

In [ ]:

%%bash

# convert single files

ifi=/Volumes/bathymetry/_blueheart/00_SONNE/SONNE_GEOMAR/SO254/SO254_products/_grd/SO254_EM122_400m_CUBE_A_EPSG3395.tif
ofi=/Volumes/bathymetry/_blueheart/00_SONNE/SONNE_GEOMAR/SO254/SO254_products/_grd/SO254_EM122_400m_CUBE_A_EPSG3395_zero.tif
shp=/Volumes/bathymetry/_blueheart/00_SONNE/SONNE_GEOMAR/SO254/SO254_products/_grd/SO254_EM122_400m_CUBE_A_EPSG3395_area.gpkg
gdal_calc.py -A $ifi --outfile=$ofi --co="COMPRESS=DEFLATE" --co="TILED=YES" --calc="(A*0)+1"
gdal_polygonize.py $ofi $shp

In [ ]:
#!/bin/bash
%%bash
cd /Volumes/bathymetry/_blueheart/00_SONNE/SONNE_GEOMAR

# Only uf neccessary: remove earlier versions of zero grids
# find . -maxdepth 4 -name '*_zero.tif' -exec rm {} \;


ifi=SONNE_grid_list.txt
IFS=$'\n'       
set -f    
for f in $(cat < "$ifi"); do
  gdal_calc.py -A "$f" --outfile="${f%%.*}_zero.tif" --calc="(A*0)+1"
done

# create list from zero grids
#find . -name "*_zero.tif" > METEOR_zero_list.txt 
ls */*/_grd/*_zero.tif > SONNE_zero_list.txt 

# polygonise zero grids to avoid millions of features due to colour change
ifi_z=SONNE_zero_list.txt
IFS=$'\n'       
set -f          
for zf in $(cat < "$ifi_z"); do
  gdal_polygonize.py "$zf" "${zf%%.*}_area.gpkg"
done

# create list from shape files
#find . -name "*_area.gpkg" > METEOR_gpkg_list.txt
ls */*/_grd/*_area.gpkg > SONNE_gpkg_list.txt
ls */*/_grd/*_EPSG3395_Area.gpkg > SONNE_gpkg_list.txt

# create list from _shp folders
#find . type dir -name "/_shp" > METEOR_shp_list.txt

# copy shape to _shp
find . -maxdepth 4 -name '*_Area.gpkg' -exec mv {} SONNE_Cov/ \;
find . -maxdepth 4 -name '*_zero_area.gpkg' -exec rm {} \;




In [ ]:
import geopandas as gpd
import pandas as pd
import os
import numpy as np

path = "/Volumes/bathymetry/_blueheart/00_SONNE/SONNE_GEOMAR"
shp_list = "//Volumes/bathymetry/_blueheart/00_SONNE/SONNE_GEOMAR/SONNE_gpkg_list.txt"
grid_list = '/Volumes/bathymetry/_blueheart/00_SONNE/SONNE_GEOMAR/SONNE_grid_list.txt'
with open(grid_list, 'r') as g_fi:
    grid_path = g_fi.readlines()
with open(shp_list, 'r') as s_fi:
    shp_path = s_fi.readlines()
    for s_path, g_path in zip(shp_path, grid_path):
        shp = os.path.join(path,s_path)
        grid = os.path.join(path,g_path)
        #shp_df = s_path.split('/')[0]
        shp_df = gpd.read_file(shp, index_col = False)
        shp_df_red = shp_df.drop(shp_df[shp_df['DN'] == 0].index)
        shp_diss = shp_df_red.dissolve(by = 'DN')
        shp_diss['Filename'] = os.path.basename(grid)
        shp_diss['Area [km2]'] = np.sum(shp_diss['geometry'].area)/(1000*1000)
        #out_shp = shp[:-15] + '_area.shp'
        out_shp = shp.replace('_zero_area.gpkg', '_Area.gpkg')
        print(f'out_shp: {out_shp}')
        shp_diss.to_file(out_shp, driver='GPKG', mode='w')
        #print(shp_df_red['DN'])


#### Remove boxes around coverages
- find out which DN has to be removed 
- DN -2147483648 für Meteor & Sonne
- DN 0 for Merian

In [ ]:
from pathlib import Path, PurePath
import geopandas as gpd
import pandas as pd
import numpy as np

path_gpkg = Path('/Volumes/bathymetry/_blueheart/00_SONNE/SONNE_GEOMAR/SONNE_Cov')
#gpkg = path_gppk.rglob('*.gpkg')
for shp in path_gpkg.glob('*.gpkg'):
    print(shp)
    shp_df = gpd.read_file(shp, index_col = False)
    shp_df_red = shp_df.drop(shp_df[shp_df['DN'] == -2147483648].index)
    shp_diss = shp_df_red.dissolve(by = 'DN')
    #out_shp = shp.replace('*_Area.gpkg', '*_Area_box.gpkg')
    out_shp = shp.with_name(f"{shp.stem.replace('Area','Area_box')}{shp.suffix}")
    print(f'out_shp: {out_shp}')
    shp_diss.to_file(out_shp, driver='GPKG', mode='w')

In [ ]:
# Merge gpkg such that only features that contain other values than 0 (mostly 1) 
%%bash

cd /Volumes/bathymetry/_blueheart/00_METEOR/METEOR_GEOMAR
mkdir /Volumes/bathymetry/_blueheart/00_METEOR/METEOR_GEOMAR/METEOR_Cov

ifi_z=METEOR_gpkg_list.txt
IFS=$'\n'      
set -f  
for zf in $(cat < "$ifi_z"); do
  mv "$zf" /Volumes/bathymetry/_blueheart/00_METEOR/METEOR_GEOMAR/METEOR_Cov
done

ogrmerge.py -f GPKG -o SONNE_GEOMAR_TID_EPSG3395.gpkg *_box.gpkg -src_geom_type MULTIPOLYGON
ogrmerge.py -f GPKG -o UnchartedSeamounts_Sectors_30.gpkg UnchartedSeamounts_Sector*.gpkg 

